# Problema 1:

Determine los desplazamientos de los bordes libres y los esfuerzos en función de la posición para la
placa empotrada mostrada en la figura. Tome las dos particiones en elementos mostradas debajo y
compare los resultados. Asuma que E = 30 x 106
 psi y n = 0.30.

Partición 1

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import copy
np.set_printoptions(precision = 4, linewidth = 150)

E = 30e6     # psi
v = 0.3      # Poisson
t = 1        # in (espesor)
h = 10       # in (altura)
l = 20       # in (largo)
T = 10000     # psi (tensión aplicada)
Fext = T / 2  # lb (fuerzas)

MN1 = [[0,0], [0,h], [l,h], [l,0], [l/2, h/2]]
MN2 = [[0,0], [0,h], [l,h], [l,0], [l/2, h/2]]

MC1 = [[0,2,1], [0,3,2]]
MC2 = [[4,0,1], [4,0,3], [4,2,3], [4,2,3]]
#CC = [[2, 0], [2, 0], [2, 0], [2, 0], [1, Fext], [1, 0], [1, Fext], [1,0]]
r = [4,5,6,7]
R = [Fext, 0, Fext, 0]
s = [0,1,2,3]
S = [0,0,0,0]

GL = 2     # Grados de libertad de un nodo


def MEF(GL, MN, MC, CC, E, v, t, h, l):
    nod = len(MN)    # cantidad de nodos
    e = len(MC)      # cantidad de elementos
    K = np.zeros([nod*GL, nod*GL])
    Esf= []
    Bs=[]
    for a in range(e):
        A = (h*l)/2
        B = np.zeros([3,6])
        c = []               #coordenadas de los nodos del elemento
        for i in MC[a]:
            c.append(MN[i])
            b=[0,0,0]
            g=[0,0,0]
        for d in range(3): 
            b[d] = c[d-2][1]- c[d-1][1]
            g[d] = c[d-1][0]- c[d-2][0]
        for j in range(3):
            B0=[[b[j],0],[0, g[j]], [g[j], b[j]]]
            B[0][j*2]=b[j]
            B[1][2*j+1]=g[j]
            B[2][j*2]=g[j]
            B[2][j*2+1]=b[j]
        B = B/(2*A)
        Bs.append(B)
        D = (E/(1-(v**2)))*np.array([[1,v,0],[v,1,0],[0,0,(1-v)/2]])
        k = t*abs(A)*np.dot(B.transpose(), np.dot(D,B) )
        K0 = np.zeros([nod*2, nod*2])
        for i in range(3):
            nx=MC[a][i]
            for j in range(3):
                ny=MC[a][j]
                K0[np.ix_([nx*2,nx*2+1], [ny*2,ny*2+1])]+=k[i*2:i*2+2, j*2:j*2+2]
        K+=K0

    d = np.zeros(nod*GL)
    f = np.zeros(nod*GL)

    #indices de Fuerzas y desplazamientos conocidxs:
    '''
    fc=[]
    dc=[]
    for i in range(len(CC)):
        if CC[i][0]==1:
            fc.append([i,CC[i][1]])
        else: 
            dc.append([i,CC[i][1]])
            d[i]=CC[i][1]

    fc=np.array(fc).astype(int)
    dc=np.array(dc)
    '''
    
    #dd = np.linalg.solve(K[np.ix_(fc[:,0], fc[:,0])], fc[:,1])
    dd = np.linalg.solve(K[np.ix_(r,r)], R)

    #for i in range(len(fc)):
     #   d[fc[i,0]]=dd[i]
    
    for i in range(len(r)):
        d[r[i]]=dd[i]

    F=K.dot(d)
    B=np.array(B)
    D=np.array(D)

    for i in range(e):
        j=np.array(MC[i])*2 
        Esf.append( np.dot( np.dot(D,Bs[i]), d[[j[0],j[0]+1,j[1],j[1]+1,j[2],j[2]+1]] ) ) 
    
    return d, Esf, F, r, s



In [23]:
# Particion 1
d1, Esf1, F1, fc, dc= MEF(GL, MN1, MC1, CC, E, v, t, h, l)
print(f'd1 = {d1/1e-6}\n Esf1 = {Esf1}\n F1 = {F1}')

d1 = [  0.       0.       0.       0.     609.581    4.1633 663.7043 104.0833   0.       0.    ]
 Esf1 = [array([1004.8038,  301.4412,    2.4019]), array([995.1962,  -1.201 ,  -2.4019])]
 F1 = [-5.0000e+03 -3.0024e+03 -5.0000e+03  3.0024e+03  5.0000e+03  1.1369e-13  5.0000e+03 -9.0949e-13  0.0000e+00  0.0000e+00]


In [24]:
# Particion 2
d2, Esf2, F2 , fc, dc= MEF(GL, MN2, MC2, CC, E, v, t, h, l)
print(f'd2 = {d2/1e-6}\n Esf2 = {Esf2}\n F2 = {F2}')

d2 = [   0.        0.        0.        0.      672.5415 -182.1408  412.7469   11.1553    0.        0.    ]
 Esf2 = [array([0., 0., 0.]), array([ 334.6596,   83.6649, -234.9053]), array([-798.8824,   50.2794, -100.5588]), array([-798.8824,   50.2794, -100.5588])]
 F2 = [ 3.3788e+02  1.6894e+02  0.0000e+00  0.0000e+00  5.0000e+03  1.1937e-12  5.0000e+03  0.0000e+00 -1.0338e+04 -1.6894e+02]
